# 1. IMDB (순환신경망) - LSTM

In [1]:
from tensorflow import keras
from keras.datasets import imdb

In [2]:
# 가장 자주 나타나는 단어 300개만 사용
(train_data, train_target), (test_data, test_target) = imdb.load_data(num_words=300)

17473536/17464789 [==============================] - 0s 0us/step


In [3]:
print(train_data.shape, train_target.shape)
print(test_data.shape, test_target.shape)

(25000,) (25000,)
(25000,) (25000,)


In [4]:
from sklearn.model_selection import train_test_split

train_data, val_data, train_target, val_target = train_test_split(train_data, train_target, test_size=0.2, random_state=0)

In [5]:
# 리뷰 길이를 100으로 맞추기
# 100보다 큰 경우 앞부분을 자르고, 100보다 작은 경우 앞부분에 0을 넣음
from keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_data, maxlen=100)
val_seq = pad_sequences(val_data, maxlen=100)
print(train_seq.shape, val_seq.shape)   # (샘플 갯수, 토큰 개수(타임스텝 개수))

(20000, 100) (5000, 100)


In [6]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Embedding(300, 5, input_length=100))  # (어휘 사전 크기, 임베딩 벡터 크기, 시퀀스 길이)
model.add(layers.LSTM(8))  # 활성화함수(기본값) : tanh
model.add(layers.Dense(1, activation='sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 5)            1500      
                                                                 
 lstm (LSTM)                 (None, 8)                 448       
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 1,957
Trainable params: 1,957
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-lstm.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

history = model.fit(train_seq, train_target, epochs=100,
                    validation_data=(val_seq, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
625/625 [==============================] - 12s 8ms/step - loss: 0.6047 - accuracy: 0.6558 - val_loss: 0.5434 - val_accuracy: 0.7214
Epoch 2/100
625/625 [==============================] - 4s 7ms/step - loss: 0.5084 - accuracy: 0.7562 - val_loss: 0.5117 - val_accuracy: 0.7494
Epoch 3/100
625/625 [==============================] - 5s 7ms/step - loss: 0.4980 - accuracy: 0.7605 - val_loss: 0.5359 - val_accuracy: 0.7270
Epoch 4/100
625/625 [==============================] - 4s 7ms/step - loss: 0.4934 - accuracy: 0.7640 - val_loss: 0.5130 - val_accuracy: 0.7446


In [9]:
test_seq = pad_sequences(test_data, maxlen=100)

In [10]:
model.evaluate(test_seq[:100], test_target[:100])

4/4 [==============================] - 0s 6ms/step - loss: 0.4629 - accuracy: 0.7500


[0.4628753364086151, 0.75]

In [11]:
# Multi-Layer RNN
model2 = models.Sequential()
model2.add(layers.Embedding(300, 5, input_length=100))  # (어휘 사전 크기, 임베딩 벡터 크기, 시퀀스 길이)
model2.add(layers.LSTM(8, return_sequences=True)) 
model2.add(layers.LSTM(8)) 
model2.add(layers.Dense(1, activation='sigmoid'))

In [12]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 5)            1500      
                                                                 
 lstm_1 (LSTM)               (None, 100, 8)            448       
                                                                 
 lstm_2 (LSTM)               (None, 8)                 544       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 2,501
Trainable params: 2,501
Non-trainable params: 0
_________________________________________________________________


In [13]:
model2.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

history = model2.fit(train_seq, train_target, epochs=100,
                     validation_data=(val_seq, val_target),
                     callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
625/625 [==============================] - 10s 12ms/step - loss: 0.5887 - accuracy: 0.6737 - val_loss: 0.5239 - val_accuracy: 0.7402
Epoch 2/100
625/625 [==============================] - 7s 12ms/step - loss: 0.5092 - accuracy: 0.7531 - val_loss: 0.5270 - val_accuracy: 0.7356
Epoch 3/100
625/625 [==============================] - 7s 12ms/step - loss: 0.5005 - accuracy: 0.7589 - val_loss: 0.5130 - val_accuracy: 0.7508
Epoch 4/100
625/625 [==============================] - 7s 12ms/step - loss: 0.4975 - accuracy: 0.7618 - val_loss: 0.5034 - val_accuracy: 0.7522
Epoch 5/100
625/625 [==============================] - 7s 12ms/step - loss: 0.4907 - accuracy: 0.7656 - val_loss: 0.5420 - val_accuracy: 0.7408
Epoch 6/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4889 - accuracy: 0.7629 - val_loss: 0.5018 - val_accuracy: 0.7500
Epoch 7/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4876 - accuracy: 0.7638 - val_loss: 0.4995 - val_accuracy

In [14]:
model2.evaluate(test_seq[:100], test_target[:100])

4/4 [==============================] - 0s 6ms/step - loss: 0.4895 - accuracy: 0.7900


[0.48950350284576416, 0.7900000214576721]